In [ ]:
def get_xi3d(cat_sdss,cat_random):
    ######################
    minsep = 90.      # Min theta
    maxsep = 140.      # Max theta
    nbins = 10        # Bins in theta
    #nbootstrap = 500  # No. of bootstrap resampling
    #brute = False     # Brute force for TreeCorr
    npatch = 10
    nthreads = 4

    config = {"min_sep": minsep, \
                "max_sep": maxsep, \
                "nbins": nbins, \
                #"sep_units": 'degree', \
                "bin_type": 'Linear', \
                #"brute": brute, \
                "metric": 'Rperp', \
                "var_method": 'jackknife', \
                #"num_bootstrap": nbootstrap,\
                "num_threads": nthreads
                }
    ######################

    import numpy as np
    import pandas as pd
    import treecorr
    from astropy.cosmology import FlatLambdaCDM
    cosmo = FlatLambdaCDM(H0=67.74, Om0=0.3089)

    def H(ra, dec, z):

        ra  = np.array(ra,  dtype=np.float32) 
        dec = np.array(dec, dtype=np.float32) 
        z   = np.array(z,   dtype=np.float32) 

        r = np.float32(cosmo.comoving_distance(z).value)
        x = r*np.cos(np.deg2rad(ra))*np.cos(np.deg2rad(dec))
        y = r*np.sin(np.deg2rad(ra))*np.cos(np.deg2rad(dec))
        z = r*np.sin(np.deg2rad(dec))
        return x, y, z

    cat_random["x"], cat_random["y"], cat_random["z"] = H(cat_random["ra"], cat_random["dec"], cat_random["red"])

    # TreeCorr Catalogues
    dd = treecorr.NNCorrelation(config)
    dr = treecorr.NNCorrelation(config)
    rr = treecorr.NNCorrelation(config)

    rcat = treecorr.Catalog(x=cat_random['x'], y=cat_random['y'], z=cat_random['z'], npatch=npatch)

    gcat = treecorr.Catalog(x=cat_sdss['x'], y=cat_sdss['y'], z=cat_sdss['z'], npatch=npatch)

    dd.process(gcat)
    rr.process(rcat)
    dr.process(gcat,rcat)

    dd.write('2dcorrelation.out')

    #xi, varxi = dd.calculateXi(rr=rr,dr=dr)
    #r = dd.meanr
    #return xi, varxi, r

import pandas as pd

# Read Gxs
zmin, zmax = 0.0, 0.15
cat_sdss = pd.read_csv('../data/sdss_zmin_0.000_zmax_0.300_sigma_5.0.csv')
cat_sdss  = cat_sdss[(cat_sdss["red"] > zmin)*(cat_sdss["red"] < zmax)]

# Filament Gxs
filgxs = cat_sdss[cat_sdss['dist_fil']<=3.]

# Non-filament Gxs
nonfilgxs = cat_sdss[cat_sdss['dist_fil']>8.25]

# Read Randoms
cat_random = pd.read_csv('../data/random_sample_healpy_128.csv')

get_xi3d(cat_sdss,cat_random)

#xi_filgxs, varxi_filgxs, r = get_xi3d(filgxs,cat_random)

#xi_nonfilgxs, varxi_nonfilgxs, r = get_xi3d(nonfilgxs,cat_random)
